In [8]:
pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.4 MB/s  0:00:01 eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 25.2
    Uninstalling pip-25.2:
      Successfully uninstalled pip-25.2
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
train_path = "/Users/pranjalsharma/Downloads/sales-forecasting/archive/train.csv"
test_path  = "/Users/pranjalsharma/Downloads/sales-forecasting/archive/test.csv"
store_path = "/Users/pranjalsharma/Downloads/sales-forecasting/archive/store.csv"


In [13]:
train = pd.read_csv(train_path)


/var/folders/lh/7d8p88ps5rx0l3scbz88rbph0000gn/T/ipykernel_4574/3098077256.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train_path)


In [14]:
test  = pd.read_csv(test_path)


In [15]:
store = pd.read_csv(store_path)

In [16]:
print(train.shape, test.shape, store.shape)
train.head()

(1017209, 9) (41088, 8) (1115, 10)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
